In [3]:
# Load libs:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from glob import glob
from os.path import dirname, abspath, join, exists

from sklearn import linear_model

from amplify.data import DataGenerator
# ^^^ For this to work you need to run `pip install -r requirements.txt`

## Load Data

In [4]:
# DataGenerator - ClearML
data_generator = DataGenerator()
xy_data = data_generator.load_data()
xy_data.head()

Info: Successfully loaded building data!


/Users/christianwelling/Fourthbrain/Fourthbrain/fourthbrain_venv/lib/python3.8/site-packages/pysolar/radiation.py:50: RuntimeWarning: overflow encountered in exp
  return flux * math.exp(-1 * optical_depth * air_mass_ratio) * is_daytime
/Users/christianwelling/Fourthbrain/Fourthbrain/fourthbrain_venv/lib/python3.8/site-packages/pysolar/radiation.py:50: RuntimeWarning: invalid value encountered in multiply
  return flux * math.exp(-1 * optical_depth * air_mass_ratio) * is_daytime


Info: Successfully loaded weather data!


,temp,pressure,humidity,clouds_all,irradiance,solar,usage
2021-10-19 04:00:00+00:00,10.07,1017.0,66.0,20.0,0.0,1.0,11.3
2021-10-19 05:00:00+00:00,9.51,1017.0,70.0,20.0,0.0,1.0,13.2
2021-10-19 06:00:00+00:00,9.28,1017.0,71.0,20.0,0.0,1.0,12.7
2021-10-19 07:00:00+00:00,9.12,1017.0,72.0,20.0,0.0,1.0,17.2
2021-10-19 08:00:00+00:00,9.12,1017.0,73.0,20.0,0.0,1.0,17.0


In [5]:
# What is the shape of the data?
print(f"XY Data Shape: {xy_data.shape}")

XY Data Shape: (1994, 7)


In [6]:
# Lets see a few more rows
xy_data.head(20)

,temp,pressure,humidity,clouds_all,irradiance,solar,usage
2021-10-19 04:00:00+00:00,10.07,1017.0,66.0,20.0,0.000000,1.0,11.3
2021-10-19 05:00:00+00:00,9.51,1017.0,70.0,20.0,0.000000,1.0,13.2
2021-10-19 06:00:00+00:00,9.28,1017.0,71.0,20.0,0.000000,1.0,12.7
2021-10-19 07:00:00+00:00,9.12,1017.0,72.0,20.0,0.000000,1.0,17.2
2021-10-19 08:00:00+00:00,9.12,1017.0,73.0,20.0,0.000000,1.0,17.0
2021-10-19 09:00:00+00:00,8.91,1017.0,75.0,1.0,0.000000,1.0,14.2
2021-10-19 10:00:00+00:00,8.61,1018.0,75.0,1.0,0.000000,1.0,14.4
2021-10-19 11:00:00+00:00,8.97,1018.0,73.0,1.0,0.000000,1.0,14.1
2021-10-19 12:00:00+00:00,9.12,1018.0,75.0,1.0,323.335022,11.3,12.5
2021-10-19 13:00:00+00:00,10.96,1019.0,69.0,1.0,680.919505,24.9,13.7


In [8]:
# timestamp example
xy_data.index[0]

Timestamp('2021-10-19 04:00:00+0000', tz='UTC', freq='H')

In [ ]:
# Data Shape 
# Batch Size of 10
# Series of 7 Days 168
# (B,  S,   H, F) <-X Y-> (B,  S,   H, O)
# (20, 30, 48, 5) <-X Y-> (20, 30, 48, 1)

## Data Processing

### Helper Functions

```
1. Make xy data into a set of series of shape [N, M, F],
where N = number of examples; M = nubmer of timestamps; F = Number of features
```

In [9]:
# TODO add random sample to stride between 1 and 5
def make_series(dataframe, series_length = 48, stride = 3):
    start_index = series_length
    end_index = len(dataframe)

    data = []
    for i in range(start_index, end_index, stride):
        indices = range(i - series_length, i, 1)
        data.append(dataframe.iloc[indices])

    return np.array(data)

In [10]:
# Create Series Data
series_data = make_series(xy_data)
series_data.shape

(649, 48, 7)

In [11]:
# Data Shape 
# Batch Size of 10
# Series of 7 Days 168
# (B,  S,   H, F) <-X Y-> (B,  S,   H, O)
# (20, 30, 48, 5) <-X Y-> (20, 30, 48, 1)

```
2. Train-Test-Validation Split. Split off at 0.8, 0.1, 0.1 ratio for now. Shuffle the split.
```

In [12]:
#TODO remove the random seed
def train_val_test_split(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True):
    assert (train_split + test_split + val_split) == 1

    if shuffle:
        # Specify seed to always have the same split distribution between runs
        rng = np.random.default_rng(seed=42)
        rng.shuffle(ds, axis=0)

    indices_or_sections = [int(train_split * ds.shape[0]), int((1 - test_split) * ds.shape[0])]
    
    train_ds, val_ds, test_ds = np.split(ds, indices_or_sections)
    
    return train_ds, val_ds, test_ds

In [14]:
# Create test, validation, and test datasets
train_ds, val_ds, test_ds = train_val_test_split(series_data)

```
3. Seperate x, y from dataset. Remove last column from dataset to extract y.
```

In [15]:
def xy_splits(dataset):
    x_ds, y_solar, y_usage = dataset[:, :, 0:-2], dataset[:, :, -2], dataset[:, :, -1]
    return x_ds, y_solar, y_usage

In [16]:
# Isolate XY from datasets
x_train, y_solar_train, y_usage_train = xy_splits(train_ds)     # Training data
x_val, y_solar_val, y_usage_val = xy_splits(val_ds)             # Validation data
x_test, y_solar_test, y_usage_test = xy_splits(test_ds)         # Test dataß

In [17]:
# format data for training...
# [1,2,....] <= was a column
# do we do (n, 1, n timesteps, n features per timestep) (3,4,5 => 2 days)?

### Reviewing Data

In [18]:
# x_train shape
print("x_train shape")
x_train.shape

x_train shape


(519, 48, 5)

In [20]:
# y_train shape
print("y_solar_train shape")
y_solar_train.shape

y_solar_train shape


(519, 48)

## Modeling

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from keras import Input
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.layers import Bidirectional
from keras.layers import TimeDistributed

### Normalize Data

In [23]:
#TODO: Move to model pipeline
# Normalize X_train data
norm_layer = tf.keras.layers.Normalization(axis=-1)
norm_layer.adapt(x_train)

2022-01-28 23:58:04.188334: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-28 23:58:04.806940: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


## Simple LSTM

In [24]:
def CreateModel(n_layer=1, n_series_len=48, n_ft=5, n_outputs=1):
    lstm_input = Input(shape=(n_series_len, n_ft))
    lstm_layer = LSTM(n_layer, activation='relu')(lstm_input)
    x = Dense(n_outputs)(lstm_layer)
    model = Model(inputs=lstm_input, outputs=x)
    return model

In [25]:
n_layer = 6
n_series_len = 48
n_ft = 5
n_outputs = 1
model = CreateModel(n_layer, n_series_len, n_ft, n_outputs)

In [26]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 5)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 6)                 288       
_________________________________________________________________
dense (Dense)                (None, 1)                 7         
Total params: 295
Trainable params: 295
Non-trainable params: 0
_________________________________________________________________


In [27]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
#model.compile(loss=keras.losses.MeanAbsoluteError(), optimizer=optimizer)
model.compile(tf.optimizers.Adam(learning_rate=0.0003),
      loss='binary_crossentropy',
      metrics=['acc'])

In [28]:
history = model.fit(x=x_train, y=y_solar_train, epochs=100, batch_size=10, validation_split=0.2, shuffle=False)

Epoch 1/100
42/42 [==============================] - 5s 52ms/step - loss: 105.9219 - acc: 0.0000e+00 - val_loss: 103.1343 - val_acc: 0.0000e+00
Epoch 2/100
42/42 [==============================] - 2s 44ms/step - loss: 105.9219 - acc: 0.0000e+00 - val_loss: 103.1343 - val_acc: 0.0000e+00
Epoch 3/100
42/42 [==============================] - 2s 40ms/step - loss: 105.9219 - acc: 0.0000e+00 - val_loss: 103.1343 - val_acc: 0.0000e+00
Epoch 4/100
42/42 [==============================] - 2s 38ms/step - loss: 105.9219 - acc: 0.0000e+00 - val_loss: 103.1343 - val_acc: 0.0000e+00
Epoch 5/100
42/42 [==============================] - 2s 41ms/step - loss: 105.9219 - acc: 0.0000e+00 - val_loss: 103.1343 - val_acc: 0.0000e+00
Epoch 6/100
42/42 [==============================] - 2s 48ms/step - loss: 105.9219 - acc: 0.0000e+00 - val_loss: 103.1343 - val_acc: 0.0000e+00
Epoch 7/100
42/42 [==============================] - 3s 62ms/step - loss: 105.9219 - acc: 0.0000e+00 - val_loss: 103.1343 - val_acc: 0.0